El problema
========

El archivo `all.BRCA-EU.mutations-context.tsv` tiene el siquiente formato:

In [19]:
import pandas as pd

# Open the file
mutations_reader = pd.read_table('~/results/all.BRCA-EU.mutations-context.tsv', 
                                 header=1, 
                                 delim_whitespace=True,
                                 iterator=True)  # Lazily load lines since file may be large
# Read a part of the file
mutations = mutations_reader.get_chunk(10)
mutations

,MUTATION_ID,MUTATION,POSITION_GRCh37,POSITION_GRCh38,RELATIVE_POSITION,OVERLAPPED_GENES,CONSEQUENCE(S),PROJECT(S)
0,MU64868974,C>G,chr2:169922536,chr2:169066026,INTRONIC,ENSG00000073737(DHRS9),"ENSG00000073737(DHRS9):intron_variant,ENSG0000...",BRCA-EU
1,MU67221130,C>T,chr2:1699231,chr2:1695459,INTRONIC,ENSG00000130508(PXDN),"ENSG00000130508(PXDN):intron_variant,ENSG00000...",BRCA-EU
2,MU64619292,T>-,chr2:16992349,chr2:16811082,INTERGENIC,BRCA-EU,NaN,NaN
3,MU66012277,C>T,chr2:169923756,chr2:169067246,NON-CODING-EXONIC,ENSG00000073737(DHRS9),"ENSG00000073737(DHRS9):intron_variant,ENSG0000...",BRCA-EU
4,MU66538864,A>T,chr2:169925859,chr2:169069349,INTRONIC,ENSG00000073737(DHRS9),"ENSG00000073737(DHRS9):intron_variant,ENSG0000...",BRCA-EU
5,MU63433685,T>A,chr2:169926262,chr2:169069752,INTRONIC,ENSG00000073737(DHRS9),"ENSG00000073737(DHRS9):intron_variant,ENSG0000...",BRCA-EU
6,MU64418202,C>G,chr2:169926829,chr2:169070319,NON-CODING-EXONIC,ENSG00000073737(DHRS9),"ENSG00000073737(DHRS9):5_prime_UTR_variant,ENS...",BRCA-EU
7,MU66019785,T>C,chr2:169927702,chr2:169071192,INTRONIC,ENSG00000073737(DHRS9),"ENSG00000073737(DHRS9):intron_variant,ENSG0000...",BRCA-EU
8,MU63584668,G>A,chr2:16992866,chr2:16811599,INTERGENIC,BRCA-EU,NaN,NaN
9,MU66013896,T>G,chr2:169928818,chr2:169072308,INTRONIC,ENSG00000073737(DHRS9),"ENSG00000073737(DHRS9):intron_variant,ENSG0000...",BRCA-EU


El archivo provee para cada mutación su posición en el ensamble genómico GRCh37 o GRCh38 y un campo `MUTATION` que especifica el alelo de referencia y el alelo mutado. El alelo de referencia es lo que se debería encontrar en la posición especificada en el ensamble genómico correspondiente.

Después de validar por dos métodos diferentes, comprobamos que en varios casos el alelo de referencia especificado en el archivo no concordaba con lo que se encuentra en el ensamble genómico en ~10% de las mutaciones, sino que se encontró que todas estas estaban desfasadas.

El problema real
=========

Después de un análisis se llegó a la raiz del problema.

El archivo `all.BRCA-EU.mutations-context.tsv` es el resultado de analizar el archivo de mutaciones de ICGC, el cual tiene el siguiente formato: 

In [20]:
# Open the file
ssmfile_reader = pd.read_table('../data/data-release-25/ssm.vcf',
                               names=['CHROM','POS','ID','REF','ALT','QUAL','FILTER','INFO'],
                               comment='#',
                               delim_whitespace=True,
                               iterator=True)  # Lazily load lines since file may be large
# Read a part of the file
ssmfile_mutations = ssmfile_reader.get_chunk(10)
ssmfile_mutations

,CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO
0,1,1000000,MU88749506,T,.,.,.,CONSEQUENCE=.;OCCURRENCE=NKTL-SG|23|23|1.00000...
1,1,100000022,MU39532371,C,T,.,.,"CONSEQUENCE=||||||intergenic_region||,RP11-413..."
2,1,100000049,MU87095619,TA,T,.,.,"CONSEQUENCE=||||||intergenic_region||,RP11-413..."
3,1,100000110,MU82202760,G,A,.,.,"CONSEQUENCE=||||||intergenic_region||,RP11-413..."
4,1,100000128,MU85052896,A,C,.,.,"CONSEQUENCE=||||||intergenic_region||,RP11-413..."
5,1,10000015,MU91785757,A,G,.,.,CONSEQUENCE=NMNAT1|ENSG00000173614|+|NMNAT1-00...
6,1,100000181,MU259333,G,A,.,.,"CONSEQUENCE=||||||intergenic_region||,RP11-413..."
7,1,100000340,MU64917116,AGGAATAGGGTGGGTCTGTGGCATTTAATCAGCGG,AGTATAGAGTGTAAAGAGT,.,.,"CONSEQUENCE=||||||intergenic_region||,RP11-413..."
8,1,100000409,MU1214865,G,A,.,.,"CONSEQUENCE=||||||intergenic_region||,RP11-413..."
9,1,100000520,MU66254099,T,C,.,.,"CONSEQUENCE=||||||intergenic_region||,RP11-413..."


Los campos a destacar de este archivo son `REF`, `INFO` y el subcampo `mutation` de `INFO`.

In [41]:
import re
mut = ssmfile_mutations.iloc[0]

# REF
print('REF: ', mut['REF'])

# INFO
print('\nINFO:', mut['INFO'])

# INFO.mutation
print('\nINFO.mutation: ',re.findall('(mutation=.*?);', mut['INFO'])[0])

REF:  T

INFO: CONSEQUENCE=.;OCCURRENCE=NKTL-SG|23|23|1.00000;affected_donors=23;mutation=T>T;project_count=1;studies=.;tested_donors=12198

INFO.mutation:  mutation=T>T


En este ejemplo, el campo `mutation` especifica que en la referencia hay una 'T', lo mismo que indica `REF` pero no siempre es así.

In [39]:
## From data release 22
! grep MU64619292 ../data/data-release-22/ssm.vcf

2	16992349	MU64619292	AT	A	.	.	CONSEQUENCE=||||||intergenic_region||;OCCURRENCE=BRCA-EU|1|560|0.00179;affected_donors=1;mutation=T>-;project_count=1;tested_donors=10638


En este ejemplo, `REF` indica que en la posición 16992349 del cromosoma 2 debe haber 'AT', y `mutation` indica un cambio de 'T' a '-'!

Esto se explica notando que `mutation` es la versión condensada de `REF`>`ALT` y sólo especifica lo importante de la mutación, no necesariamente significa que si `mutation` es 'T>-', en esa posición hay una 'T'. `REF` y `ALT` son los que sirven para ubicar las mutaciones en la referencia y ellos si especifican lo que se encuentra en la referencia en la posición especificada.

Regresando al archivo en cuestión, `all.BRCA-EU.mutations-context.tsv` el problema es que a la hora de hacer el análisis no se conocía este detalle y se utilizó lo que venía en el subcampo `mutation` cuando deberían de haberse utilizado los campos `REF` y `ALT`, por lo que las mutaciones pueden estar desfasadas.

Como ejemplo, mostramos la mutación de arriba como se encuentra en `all.BRCA-EU.mutations-context.tsv`:

In [40]:
## From the analysis file
! grep MU64619292 ~/results/all.BRCA-EU.mutations-context.tsv

MU64619292	T>-	chr2:16992349	chr2:16811082	INTERGENIC			BRCA-EU	
